# Functions

In [1]:
import pandas as pd
import requests
from typing import Dict, List
import re
import json

def get_github_folder_files(username: str, repo_name: str, folder_path: str,
                           file_extension: str = ".csv") -> List[str]:
    """
    GitHub API를 사용해서 특정 폴더의 파일 목록을 가져오는 함수

    Parameters:
    -----------
    username : str
        GitHub 사용자명
    repo_name : str
        레포지토리 이름
    folder_path : str
        폴더 경로 (예: "temp/v1_g/3rd_2002")
    file_extension : str
        필터링할 파일 확장자 (기본값: ".csv")

    Returns:
    --------
    List[str]
        해당 폴더의 CSV 파일명 목록
    """

    # GitHub API URL
    api_url = f"https://api.github.com/repos/{username}/{repo_name}/contents/{folder_path}"

    try:
        response = requests.get(api_url)
        response.raise_for_status()

        files_data = response.json()

        # CSV 파일만 필터링
        csv_files = []
        for file_info in files_data:
            if file_info['type'] == 'file' and file_info['name'].endswith(file_extension):
                csv_files.append(file_info['name'])

        return sorted(csv_files)

    except requests.exceptions.RequestException as e:
        print(f"❌ GitHub API 요청 실패: {e}")
        return []
    except json.JSONDecodeError as e:
        print(f"❌ JSON 파싱 실패: {e}")
        return []


def create_variable_name_from_filename(filename: str) -> str:
    """
    파일명에서 변수명을 생성하는 함수
    temp1_governor_지역명_숫자.csv → g숫자_지역명

    Parameters:
    -----------
    filename : str
        파일명 (예: "temp1_governor_busan_3.csv", "temp1_governor_seoul_4.csv")

    Returns:
    --------
    str
        변수명 (예: "g3_busan", "g4_seoul")
    """

    # temp1_governor_지역명_숫자.csv → g숫자_지역명
    match = re.search(r'temp1_governor_([^_]+)_(\d+)\.csv', filename)
    if match:
        region = match.group(1)
        number = match.group(2)
        return f"g{number}_{region}"

    # 매칭되지 않으면 기본적으로 파일명 정리해서 반환
    base_name = filename.replace('.csv', '').replace('temp1_', '')
    return base_name


def load_all_csvs_from_github_folder(username: str, repo_name: str, folder_path: str) -> Dict[str, pd.DataFrame]:
    """
    GitHub 폴더에서 모든 CSV 파일을 자동으로 읽어와서 DataFrame 딕셔너리로 반환

    Parameters:
    -----------
    username : str
        GitHub 사용자명
    repo_name : str
        레포지토리 이름 (예: "korean-elections")
    folder_path : str
        폴더 경로 (예: "temp/v1_g/3rd_2002")

    Returns:
    --------
    Dict[str, pd.DataFrame]
        변수명을 키로 하고 DataFrame을 값으로 하는 딕셔너리
    """

    # 1. GitHub API로 파일 목록 가져오기
    csv_files = get_github_folder_files(username, repo_name, folder_path)

    if not csv_files:
        print("❌ CSV 파일을 찾을 수 없습니다.")
        return {}

    # 2. 각 CSV 파일 읽어오기
    base_url = f"https://raw.githubusercontent.com/{username}/{repo_name}/refs/heads/main/{folder_path}/"
    dataframes = {}

    for filename in csv_files:
        try:
            # URL 생성
            url = base_url + filename

            # 변수명 생성
            var_name = create_variable_name_from_filename(filename)

            # CSV 읽기
            df = pd.read_csv(url)

            dataframes[var_name] = df
            print(f"✅ {filename} → {var_name}")

        except Exception as e:
            print(f"❌ {filename} 읽기 실패: {e}")

    return dataframes


def create_individual_variables(dataframes: Dict[str, pd.DataFrame], globals_dict):
    """
    딕셔너리로 저장된 DataFrame들을 개별 변수로 생성

    Parameters:
    -----------
    dataframes : Dict[str, pd.DataFrame]
        load_all_csvs_from_github_folder()의 반환값
    globals_dict : dict
        변수를 저장할 전역 네임스페이스 (globals()를 전달)
    """

    for var_name, df in dataframes.items():
        globals_dict[var_name] = df
        print(f"변수 생성: {var_name}")


# # 사용 예시
# def example_usage():
#     """사용 예시"""

#     print("=== GitHub 폴더 자동 CSV 로더 ===\n")

#     print("사용법:")
#     print("""
#     username = "your-username"
#     repo_name = "korean-elections"
#     folder_path = "temp/v1_g/3rd_2002"

#     # 모든 CSV 파일을 자동으로 읽어와서 딕셔너리로 저장
#     election_data = load_all_csvs_from_github_folder(username, repo_name, folder_path)

#     # 방법 1: 딕셔너리로 접근
#     busan_df = election_data['g3_busan']
#     seoul_df = election_data['g3_seoul']

#     # 방법 2: 개별 변수로 생성
#     create_individual_variables(election_data, globals())
#     # 이제 g3_busan, g3_seoul 등을 직접 사용 가능
#     """)

#     print("\n다양한 번호 지원:")
#     print("""
#     # 3회차: temp1_governor_busan_3.csv → g3_busan
#     election_3rd = load_all_csvs_from_github_folder(username, repo_name, "temp/v1_g/3rd_2002")

#     # 4회차: temp1_governor_seoul_4.csv → g4_seoul
#     election_4th = load_all_csvs_from_github_folder(username, repo_name, "temp/v1_g/4th_2006")

#     # 5회차: temp1_governor_daegu_5.csv → g5_daegu
#     election_5th = load_all_csvs_from_github_folder(username, repo_name, "temp/v1_g/5th_2010")
#     """)


# if __name__ == "__main__":
#     example_usage()

# ### 간단한 사용법

# # 한 줄로 모든 CSV 읽기
# username = "your-github-username"
# repo_name = "korean-elections"
# folder_path = "temp/v1_g/3rd_2002"

# election_data = load_all_csvs_from_github_folder(username, repo_name, folder_path)

# # 사용하기
# busan_df = election_data['g3_busan']
# seoul_df = election_data['g3_seoul']

# # 또는 개별 변수로 생성
# create_individual_variables(election_data, globals())
# # 이제 g3_busan, g3_seoul 직접 사용 가능

In [2]:
import pandas as pd

def show_column_info(df):
    """
    데이터프레임의 열 이름과 인덱스를 확인하는 함수
    """
    print("=== 열 이름과 인덱스 ===")
    for idx, col_name in enumerate(df.columns):
        print(f"{idx}: {col_name}")
    print(f"\n총 열 개수: {len(df.columns)}")
    print(f"총 행 개수: {len(df)}")

def rename_columns_by_index(df, index_or_indices, new_name_or_names):
    """
    인덱스를 통해 열 이름을 변경하는 함수

    Parameters:
    - df: pandas DataFrame
    - index_or_indices: 단일 인덱스(int) 또는 인덱스 리스트([int, int, ...])
    - new_name_or_names: 단일 새 이름(str) 또는 새 이름 리스트([str, str, ...])

    Returns:
    - 열 이름이 변경된 DataFrame
    """
    # 단일 값을 리스트로 변환
    if isinstance(index_or_indices, int):
        indices = [index_or_indices]
    else:
        indices = index_or_indices

    if isinstance(new_name_or_names, str):
        new_names = [new_name_or_names]
    else:
        new_names = new_name_or_names

    # 길이 검증
    if len(indices) != len(new_names):
        raise ValueError(f"인덱스 개수({len(indices)})와 새 이름 개수({len(new_names)})가 일치하지 않습니다.")

    # 인덱스 유효성 검사
    for idx in indices:
        if idx < 0 or idx >= len(df.columns):
            raise ValueError(f"유효하지 않은 인덱스: {idx}. 유효 범위: 0 ~ {len(df.columns) - 1}")

    # 데이터프레임 복사
    result_df = df.copy()

    # 열 이름 변경
    rename_dict = {}
    for idx, new_name in zip(indices, new_names):
        old_name = df.columns[idx]
        rename_dict[old_name] = new_name
        print(f"인덱스 {idx}: '{old_name}' -> '{new_name}'")

    result_df = result_df.rename(columns=rename_dict)

    print(f"\n=== 열 이름 변경 완료 ===")
    print(f"변경된 열 개수: {len(rename_dict)}")

    return result_df

def add_sum_column(df, start_idx, end_idx, new_column_name):
    """
    연속된 인덱스 범위의 열들을 합산하여 새로운 열을 추가하는 함수

    Parameters:
    - df: pandas DataFrame
    - start_idx: 시작 인덱스
    - end_idx: 끝 인덱스 (포함)
    - new_column_name: 새로 추가할 열의 이름

    Returns:
    - 새로운 열이 추가된 DataFrame
    """
    # 유효성 검사
    if start_idx < 0 or end_idx >= len(df.columns) or start_idx > end_idx:
        raise ValueError(f"유효하지 않은 인덱스 범위: ({start_idx}, {end_idx}). 유효 범위: 0 ~ {len(df.columns) - 1}")

    # 합산할 열 이름들 추출
    columns_to_sum = df.columns[start_idx:end_idx + 1]

    print(f"\n=== 합산할 열들 (인덱스 {start_idx}~{end_idx}) ===")
    for idx, col_name in enumerate(columns_to_sum):
        print(f"{start_idx + idx}: {col_name}")

    # 데이터프레임 복사
    result_df = df.copy()

    # 지정된 열들을 합산하여 새로운 열 추가
    # 숫자가 아닌 값은 0으로 처리
    result_df[new_column_name] = df[columns_to_sum].select_dtypes(include=['number']).sum(axis=1, skipna=True)

    print(f"\n=== 새로운 열 추가 완료 ===")
    print(f"새로운 열 이름: {new_column_name}")
    print(f"총 열 개수: {len(result_df.columns)}")

    return result_df

def add_zero_column(df, column_name):
    """
    DataFrame 마지막에 모든 원소가 0(int)인 열을 추가하는 함수

    Parameters:
    - df: pandas DataFrame
    - column_name: 추가할 열의 이름 (str)

    Returns:
    - 새로운 열이 추가된 DataFrame
    """
    result_df = df.copy()
    result_df[column_name] = 0
    return result_df

# # 사용 예시
# if __name__ == "__main__":
#     # CSV 파일 읽기
#     df = pd.read_csv('temp1_president_14.csv')

#     # 열 정보 확인
#     show_column_info(df)

#     # 예시 1: 단일 열 이름 변경
#     print("\n" + "="*50)
#     print("예시 1: 단일 열 이름 변경")
#     print("="*50)
#     df_renamed = rename_columns_by_index(df, 0, '광역시도')

#     # 예시 2: 복수 열 이름 변경
#     print("\n" + "="*50)
#     print("예시 2: 복수 열 이름 변경")
#     print("="*50)
#     df_renamed = rename_columns_by_index(df_renamed, [1, 2], ['시군구', '유권자수'])

#     # 예시 3: 인덱스 4~7 열들 합산 (득표수 1~4번 후보)
#     print("\n" + "="*50)
#     print("예시 3: 열 합산")
#     print("="*50)
#     result_df = add_sum_column(df_renamed, 4, 7, '상위4후보합계')

#     # 변경된 열 정보 확인
#     print("\n" + "="*50)
#     print("변경된 열 정보")
#     print("="*50)
#     show_column_info(result_df)

#     # 결과 확인 (첫 번째 행)
#     print("\n=== 첫 번째 행 결과 ===")
#     first_row = result_df.iloc[0]
#     print(f"광역시도: {first_row['광역시도']}")
#     print(f"시군구: {first_row['시군구']}")
#     print(f"유권자수: {first_row['유권자수']}")
#     print(f"득표수_1_민주자유당_김영삼: {first_row['득표수_1_민주자유당_김영삼']}")
#     print(f"득표수_2_민주당_김대중: {first_row['득표수_2_민주당_김대중']}")
#     print(f"득표수_3_통일국민당_정주영: {first_row['득표수_3_통일국민당_정주영']}")
#     print(f"득표수_4_새한국당_이종찬: {first_row['득표수_4_새한국당_이종찬']}")
#     print(f"상위4후보합계: {first_row['상위4후보합계']}")

#     # 검증: 수동 계산과 비교
#     manual_sum = (first_row['득표수_1_민주자유당_김영삼'] +
#                   first_row['득표수_2_민주당_김대중'] +
#                   first_row['득표수_3_통일국민당_정주영'] +
#                   first_row['득표수_4_새한국당_이종찬'])

#     print(f"\n수동 계산 결과: {manual_sum}")
#     print(f"함수 계산 결과: {first_row['상위4후보합계']}")
#     print(f"계산 일치: {manual_sum == first_row['상위4후보합계']}")

#     # 결과 데이터프레임 출력 (상위 5행)
#     print("\n=== 결과 데이터프레임 (상위 5행) ===")
#     print(result_df.head())

# Data (v1, 4th_2006)

In [3]:
# 한 줄로 모든 CSV 읽기
username = "sw1kwon"
repo_name = "korean-elections"
folder_path = "temp/v1_g/4th_2006"

governor_4 = load_all_csvs_from_github_folder(username, repo_name, folder_path)

# 개별 변수로 생성
create_individual_variables(governor_4, globals())
# 이제 g4_busan, g4_seoul 직접 사용 가능

✅ temp1_governor_busan_4.csv → g4_busan
✅ temp1_governor_chungbuk_4.csv → g4_chungbuk
✅ temp1_governor_chungnam_4.csv → g4_chungnam
✅ temp1_governor_daegu_4.csv → g4_daegu
✅ temp1_governor_daejeon_4.csv → g4_daejeon
✅ temp1_governor_gangwon_4.csv → g4_gangwon
✅ temp1_governor_gwangju_4.csv → g4_gwangju
✅ temp1_governor_gyeongbuk_4.csv → g4_gyeongbuk
✅ temp1_governor_gyeonggi_4.csv → g4_gyeonggi
✅ temp1_governor_gyeongnam_4.csv → g4_gyeongnam
✅ temp1_governor_incheon_4.csv → g4_incheon
✅ temp1_governor_jeju_4.csv → g4_jeju
✅ temp1_governor_jeonbuk_4.csv → g4_jeonbuk
✅ temp1_governor_jeonnam_4.csv → g4_jeonnam
✅ temp1_governor_seoul_4.csv → g4_seoul
✅ temp1_governor_ulsan_4.csv → g4_ulsan
변수 생성: g4_busan
변수 생성: g4_chungbuk
변수 생성: g4_chungnam
변수 생성: g4_daegu
변수 생성: g4_daejeon
변수 생성: g4_gangwon
변수 생성: g4_gwangju
변수 생성: g4_gyeongbuk
변수 생성: g4_gyeonggi
변수 생성: g4_gyeongnam
변수 생성: g4_incheon
변수 생성: g4_jeju
변수 생성: g4_jeonbuk
변수 생성: g4_jeonnam
변수 생성: g4_seoul
변수 생성: g4_ulsan


## Seoul

In [4]:
g4_seoul.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_강금실,득표수_2_한나라당_오세훈,득표수_3_민주당_박주선,득표수_4_민주노동당_김종철,득표수_5_국민중심당_임웅균,득표수_6_시민당_이귀선,득표수_7_한국의미래를준비하는당_이태희,득표수_8_무소속_백승원,득표수_계,무효투표수,기권수
0,서울특별시,합계,7983648,3977842,1077890,2409760,304565,117421,14111,4790,4481,13808,3946826,31016,4005806
1,서울특별시,종로구,134603,71240,19369,41992,6460,1945,236,110,89,265,70466,774,63363
2,서울특별시,중구,107164,55369,15382,32708,4609,1455,186,193,113,212,54858,511,51795
3,서울특별시,용산구,187461,95809,26087,58427,6970,2632,318,152,124,319,95029,780,91652
4,서울특별시,성동구,266956,131927,35842,77631,11971,3975,461,165,153,488,130686,1241,135029


In [5]:
show_column_info(g4_seoul)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_열린우리당_강금실
5: 득표수_2_한나라당_오세훈
6: 득표수_3_민주당_박주선
7: 득표수_4_민주노동당_김종철
8: 득표수_5_국민중심당_임웅균
9: 득표수_6_시민당_이귀선
10: 득표수_7_한국의미래를준비하는당_이태희
11: 득표수_8_무소속_백승원
12: 득표수_계
13: 무효투표수
14: 기권수

총 열 개수: 15
총 행 개수: 26


### v2.1

In [6]:
g4_seoul_21 = rename_columns_by_index(g4_seoul, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_강금실' -> '진보정당'
인덱스 5: '득표수_2_한나라당_오세훈' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [7]:
g4_seoul_21 = add_sum_column(g4_seoul_21, 6, 11, '그외정당')


=== 합산할 열들 (인덱스 6~11) ===
6: 득표수_3_민주당_박주선
7: 득표수_4_민주노동당_김종철
8: 득표수_5_국민중심당_임웅균
9: 득표수_6_시민당_이귀선
10: 득표수_7_한국의미래를준비하는당_이태희
11: 득표수_8_무소속_백승원

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 16


In [8]:
g4_seoul_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_민주당_박주선,득표수_4_민주노동당_김종철,득표수_5_국민중심당_임웅균,득표수_6_시민당_이귀선,득표수_7_한국의미래를준비하는당_이태희,득표수_8_무소속_백승원,득표수_계,무효투표수,기권수,그외정당
0,서울특별시,합계,7983648,3977842,1077890,2409760,304565,117421,14111,4790,4481,13808,3946826,31016,4005806,459176
1,서울특별시,종로구,134603,71240,19369,41992,6460,1945,236,110,89,265,70466,774,63363,9105
2,서울특별시,중구,107164,55369,15382,32708,4609,1455,186,193,113,212,54858,511,51795,6768


In [9]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g4_seoul_21 = g4_seoul_21[selected_columns].copy()

g4_seoul_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,서울특별시,합계,7983648,3977842,2409760,1077890,459176,3946826,31016,4005806
1,서울특별시,종로구,134603,71240,41992,19369,9105,70466,774,63363
2,서울특별시,중구,107164,55369,32708,15382,6768,54858,511,51795


In [10]:
g4_seoul_21.to_csv("temp2_1_governor_seoul_4.csv", index=False, encoding="utf-8-sig")

### v2.2

In [11]:
g4_seoul_22 = rename_columns_by_index(g4_seoul, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_강금실' -> '진보정당'
인덱스 5: '득표수_2_한나라당_오세훈' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [12]:
g4_seoul_22 = add_sum_column(g4_seoul_22, 6, 10, '그외정당')
g4_seoul_22 = add_sum_column(g4_seoul_22, 11, 11, '무소속')


=== 합산할 열들 (인덱스 6~10) ===
6: 득표수_3_민주당_박주선
7: 득표수_4_민주노동당_김종철
8: 득표수_5_국민중심당_임웅균
9: 득표수_6_시민당_이귀선
10: 득표수_7_한국의미래를준비하는당_이태희

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 16

=== 합산할 열들 (인덱스 11~11) ===
11: 득표수_8_무소속_백승원

=== 새로운 열 추가 완료 ===
새로운 열 이름: 무소속
총 열 개수: 17


In [13]:
g4_seoul_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_민주당_박주선,득표수_4_민주노동당_김종철,득표수_5_국민중심당_임웅균,득표수_6_시민당_이귀선,득표수_7_한국의미래를준비하는당_이태희,득표수_8_무소속_백승원,득표수_계,무효투표수,기권수,그외정당,무소속
0,서울특별시,합계,7983648,3977842,1077890,2409760,304565,117421,14111,4790,4481,13808,3946826,31016,4005806,445368,13808
1,서울특별시,종로구,134603,71240,19369,41992,6460,1945,236,110,89,265,70466,774,63363,8840,265
2,서울특별시,중구,107164,55369,15382,32708,4609,1455,186,193,113,212,54858,511,51795,6556,212


In [14]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g4_seoul_22 = g4_seoul_22[selected_columns].copy()

g4_seoul_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,서울특별시,합계,7983648,3977842,2409760,1077890,445368,13808,3946826,31016,4005806
1,서울특별시,종로구,134603,71240,41992,19369,8840,265,70466,774,63363
2,서울특별시,중구,107164,55369,32708,15382,6556,212,54858,511,51795


In [15]:
g4_seoul_22.to_csv("temp2_2_governor_seoul_4.csv", index=False, encoding="utf-8-sig")

## Busan

In [16]:
g4_busan.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_오거돈,득표수_2_한나라당_허남식,득표수_4_민주노동당_김석준,득표수_계,무효투표수,기권수
0,부산광역시,합계,2845104,1378618,329470,895214,141061,1365745,12873,1466486
1,부산광역시,중구,43062,22768,5155,15325,2047,22527,241,20294
2,부산광역시,서구,112683,55046,11782,37607,5038,54427,619,57637
3,부산광역시,동구,89864,46773,10749,30690,4592,46031,742,43091
4,부산광역시,영도구,131447,63268,17638,38017,6857,62512,756,68179


In [17]:
show_column_info(g4_busan)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_열린우리당_오거돈
5: 득표수_2_한나라당_허남식
6: 득표수_4_민주노동당_김석준
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 17


### v2.1

In [18]:
g4_busan_21 = rename_columns_by_index(g4_busan, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_오거돈' -> '진보정당'
인덱스 5: '득표수_2_한나라당_허남식' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [19]:
g4_busan_21 = add_sum_column(g4_busan_21, 6, 6, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_4_민주노동당_김석준

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [20]:
g4_busan_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_4_민주노동당_김석준,득표수_계,무효투표수,기권수,그외정당
0,부산광역시,합계,2845104,1378618,329470,895214,141061,1365745,12873,1466486,141061
1,부산광역시,중구,43062,22768,5155,15325,2047,22527,241,20294,2047
2,부산광역시,서구,112683,55046,11782,37607,5038,54427,619,57637,5038


In [21]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g4_busan_21 = g4_busan_21[selected_columns].copy()

g4_busan_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,부산광역시,합계,2845104,1378618,895214,329470,141061,1365745,12873,1466486
1,부산광역시,중구,43062,22768,15325,5155,2047,22527,241,20294
2,부산광역시,서구,112683,55046,37607,11782,5038,54427,619,57637


In [22]:
g4_busan_21.to_csv("temp2_1_governor_busan_4.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [23]:
g4_busan_22 = rename_columns_by_index(g4_busan, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_오거돈' -> '진보정당'
인덱스 5: '득표수_2_한나라당_허남식' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [24]:
g4_busan_22 = add_sum_column(g4_busan_22, 6, 6, '그외정당')
g4_busan_22 = add_zero_column(g4_busan_22, '무소속')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_4_민주노동당_김석준

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [25]:
g4_busan_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_4_민주노동당_김석준,득표수_계,무효투표수,기권수,그외정당,무소속
0,부산광역시,합계,2845104,1378618,329470,895214,141061,1365745,12873,1466486,141061,0
1,부산광역시,중구,43062,22768,5155,15325,2047,22527,241,20294,2047,0
2,부산광역시,서구,112683,55046,11782,37607,5038,54427,619,57637,5038,0


In [26]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g4_busan_22 = g4_busan_22[selected_columns].copy()

g4_busan_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,부산광역시,합계,2845104,1378618,895214,329470,141061,0,1365745,12873,1466486
1,부산광역시,중구,43062,22768,15325,5155,2047,0,22527,241,20294
2,부산광역시,서구,112683,55046,37607,11782,5038,0,54427,619,57637


In [27]:
g4_busan_22.to_csv("temp2_2_governor_busan_4.csv", index=False, encoding="utf-8-sig")

## Daegu

In [28]:
g4_daegu.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_이재용,득표수_2_한나라당_김범일,득표수_4_민주노동당_이연재,득표수_5_국민중심당_박승국,득표수_6_무소속_백승홍,득표수_계,무효투표수,기권수
0,대구광역시,합계,1885043,915060,191131,636057,35497,8764,35232,906681,8379,969983
1,대구광역시,중구,66126,35274,7753,24056,960,228,1870,34867,407,30852
2,대구광역시,동구,267710,129221,27773,90128,4935,1291,3714,127841,1380,138489
3,대구광역시,서구,194812,92272,16137,63241,3304,807,7757,91246,1026,102540
4,대구광역시,남구,145954,70026,17919,47545,1876,387,1706,69433,593,75928


In [29]:
show_column_info(g4_daegu)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_열린우리당_이재용
5: 득표수_2_한나라당_김범일
6: 득표수_4_민주노동당_이연재
7: 득표수_5_국민중심당_박승국
8: 득표수_6_무소속_백승홍
9: 득표수_계
10: 무효투표수
11: 기권수

총 열 개수: 12
총 행 개수: 9


### v2.1

In [30]:
g4_daegu_21 = rename_columns_by_index(g4_daegu, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_이재용' -> '진보정당'
인덱스 5: '득표수_2_한나라당_김범일' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [31]:
g4_daegu_21 = add_sum_column(g4_daegu_21, 6, 8, '그외정당')


=== 합산할 열들 (인덱스 6~8) ===
6: 득표수_4_민주노동당_이연재
7: 득표수_5_국민중심당_박승국
8: 득표수_6_무소속_백승홍

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 13


In [32]:
g4_daegu_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_4_민주노동당_이연재,득표수_5_국민중심당_박승국,득표수_6_무소속_백승홍,득표수_계,무효투표수,기권수,그외정당
0,대구광역시,합계,1885043,915060,191131,636057,35497,8764,35232,906681,8379,969983,79493
1,대구광역시,중구,66126,35274,7753,24056,960,228,1870,34867,407,30852,3058
2,대구광역시,동구,267710,129221,27773,90128,4935,1291,3714,127841,1380,138489,9940


In [33]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g4_daegu_21 = g4_daegu_21[selected_columns].copy()

g4_daegu_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,대구광역시,합계,1885043,915060,636057,191131,79493,906681,8379,969983
1,대구광역시,중구,66126,35274,24056,7753,3058,34867,407,30852
2,대구광역시,동구,267710,129221,90128,27773,9940,127841,1380,138489


In [34]:
g4_daegu_21.to_csv("temp2_1_governor_daegu_4.csv", index=False, encoding="utf-8-sig")

### v2.2

In [35]:
g4_daegu_22 = rename_columns_by_index(g4_daegu, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_이재용' -> '진보정당'
인덱스 5: '득표수_2_한나라당_김범일' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [36]:
g4_daegu_22 = add_sum_column(g4_daegu_22, 6, 7, '그외정당')
g4_daegu_22 = add_sum_column(g4_daegu_22, 8, 8, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_4_민주노동당_이연재
7: 득표수_5_국민중심당_박승국

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 13

=== 합산할 열들 (인덱스 8~8) ===
8: 득표수_6_무소속_백승홍

=== 새로운 열 추가 완료 ===
새로운 열 이름: 무소속
총 열 개수: 14


In [37]:
g4_daegu_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_4_민주노동당_이연재,득표수_5_국민중심당_박승국,득표수_6_무소속_백승홍,득표수_계,무효투표수,기권수,그외정당,무소속
0,대구광역시,합계,1885043,915060,191131,636057,35497,8764,35232,906681,8379,969983,44261,35232
1,대구광역시,중구,66126,35274,7753,24056,960,228,1870,34867,407,30852,1188,1870
2,대구광역시,동구,267710,129221,27773,90128,4935,1291,3714,127841,1380,138489,6226,3714


In [38]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g4_daegu_22 = g4_daegu_22[selected_columns].copy()

g4_daegu_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,대구광역시,합계,1885043,915060,636057,191131,44261,35232,906681,8379,969983
1,대구광역시,중구,66126,35274,24056,7753,1188,1870,34867,407,30852
2,대구광역시,동구,267710,129221,90128,27773,6226,3714,127841,1380,138489


In [39]:
g4_daegu_22.to_csv("temp2_2_governor_daegu_4.csv", index=False, encoding="utf-8-sig")

## Incheon

In [40]:
g4_incheon.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_최기선,득표수_2_한나라당_안상수,득표수_3_민주당_신경철,득표수_4_민주노동당_김성진,득표수_계,무효투표수,기권수
0,인천광역시,합계,1940403,859506,200650,526932,44339,78898,850819,8687,1080897
1,인천광역시,중구,72876,34796,8119,21570,1877,2764,34330,466,38080
2,인천광역시,동구,61515,30794,6663,18707,1483,3514,30367,427,30721
3,인천광역시,남구,327614,140581,31036,90299,6402,11403,139140,1441,187033
4,인천광역시,연수구,191076,88010,19370,57235,2892,7873,87370,640,103066


In [41]:
show_column_info(g4_incheon)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_열린우리당_최기선
5: 득표수_2_한나라당_안상수
6: 득표수_3_민주당_신경철
7: 득표수_4_민주노동당_김성진
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 11


### v2.1

In [42]:
g4_incheon_21 = rename_columns_by_index(g4_incheon, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_최기선' -> '진보정당'
인덱스 5: '득표수_2_한나라당_안상수' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [43]:
g4_incheon_21 = add_sum_column(g4_incheon_21, 6, 7, '그외정당')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_민주당_신경철
7: 득표수_4_민주노동당_김성진

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [44]:
g4_incheon_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_민주당_신경철,득표수_4_민주노동당_김성진,득표수_계,무효투표수,기권수,그외정당
0,인천광역시,합계,1940403,859506,200650,526932,44339,78898,850819,8687,1080897,123237
1,인천광역시,중구,72876,34796,8119,21570,1877,2764,34330,466,38080,4641
2,인천광역시,동구,61515,30794,6663,18707,1483,3514,30367,427,30721,4997


In [45]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g4_incheon_21 = g4_incheon_21[selected_columns].copy()

g4_incheon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,인천광역시,합계,1940403,859506,526932,200650,123237,850819,8687,1080897
1,인천광역시,중구,72876,34796,21570,8119,4641,34330,466,38080
2,인천광역시,동구,61515,30794,18707,6663,4997,30367,427,30721


In [46]:
g4_incheon_21.to_csv("temp2_1_governor_incheon_4.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [47]:
g4_incheon_22 = rename_columns_by_index(g4_incheon, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_최기선' -> '진보정당'
인덱스 5: '득표수_2_한나라당_안상수' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [48]:
g4_incheon_22 = add_sum_column(g4_incheon_22, 6, 7, '그외정당')
g4_incheon_22 = add_zero_column(g4_incheon_22, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_민주당_신경철
7: 득표수_4_민주노동당_김성진

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [49]:
g4_incheon_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_민주당_신경철,득표수_4_민주노동당_김성진,득표수_계,무효투표수,기권수,그외정당,무소속
0,인천광역시,합계,1940403,859506,200650,526932,44339,78898,850819,8687,1080897,123237,0
1,인천광역시,중구,72876,34796,8119,21570,1877,2764,34330,466,38080,4641,0
2,인천광역시,동구,61515,30794,6663,18707,1483,3514,30367,427,30721,4997,0


In [50]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g4_incheon_22 = g4_incheon_22[selected_columns].copy()

g4_incheon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,인천광역시,합계,1940403,859506,526932,200650,123237,0,850819,8687,1080897
1,인천광역시,중구,72876,34796,21570,8119,4641,0,34330,466,38080
2,인천광역시,동구,61515,30794,18707,6663,4997,0,30367,427,30721


In [51]:
g4_incheon_22.to_csv("temp2_2_governor_incheon_4.csv", index=False, encoding="utf-8-sig")

## Gwangju

In [52]:
g4_gwangju.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_조영택,득표수_2_한나라당_한영,득표수_3_민주당_박광태,득표수_4_민주노동당_오병윤,득표수_계,무효투표수,기권수
0,광주광역시,합계,1014620,469316,157756,18461,239884,48617,464718,4598,545304
1,광주광역시,동구,90767,43236,16085,2201,21204,3240,42730,506,47531
2,광주광역시,서구,221034,102566,34849,3911,51556,11386,101702,864,118468
3,광주광역시,남구,161406,78309,26536,3011,42078,5861,77486,823,83097
4,광주광역시,북구,333413,147702,53265,5905,72580,14636,146386,1316,185711


In [53]:
show_column_info(g4_gwangju)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_열린우리당_조영택
5: 득표수_2_한나라당_한영
6: 득표수_3_민주당_박광태
7: 득표수_4_민주노동당_오병윤
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 6


### v2.1

In [54]:
g4_gwangju_21 = rename_columns_by_index(g4_gwangju, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_조영택' -> '진보정당'
인덱스 5: '득표수_2_한나라당_한영' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [55]:
g4_gwangju_21 = add_sum_column(g4_gwangju_21, 6, 7, '그외정당')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_민주당_박광태
7: 득표수_4_민주노동당_오병윤

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [56]:
g4_gwangju_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_민주당_박광태,득표수_4_민주노동당_오병윤,득표수_계,무효투표수,기권수,그외정당
0,광주광역시,합계,1014620,469316,157756,18461,239884,48617,464718,4598,545304,288501
1,광주광역시,동구,90767,43236,16085,2201,21204,3240,42730,506,47531,24444
2,광주광역시,서구,221034,102566,34849,3911,51556,11386,101702,864,118468,62942


In [57]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g4_gwangju_21 = g4_gwangju_21[selected_columns].copy()

g4_gwangju_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,광주광역시,합계,1014620,469316,18461,157756,288501,464718,4598,545304
1,광주광역시,동구,90767,43236,2201,16085,24444,42730,506,47531
2,광주광역시,서구,221034,102566,3911,34849,62942,101702,864,118468


In [58]:
g4_gwangju_21.to_csv("temp2_1_governor_gwangju_4.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [59]:
g4_gwangju_22 = rename_columns_by_index(g4_gwangju, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_조영택' -> '진보정당'
인덱스 5: '득표수_2_한나라당_한영' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [60]:
g4_gwangju_22 = add_sum_column(g4_gwangju_22, 6, 7, '그외정당')
g4_gwangju_22 = add_zero_column(g4_gwangju_22, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_민주당_박광태
7: 득표수_4_민주노동당_오병윤

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [61]:
g4_gwangju_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_민주당_박광태,득표수_4_민주노동당_오병윤,득표수_계,무효투표수,기권수,그외정당,무소속
0,광주광역시,합계,1014620,469316,157756,18461,239884,48617,464718,4598,545304,288501,0
1,광주광역시,동구,90767,43236,16085,2201,21204,3240,42730,506,47531,24444,0
2,광주광역시,서구,221034,102566,34849,3911,51556,11386,101702,864,118468,62942,0


In [62]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g4_gwangju_22 = g4_gwangju_22[selected_columns].copy()

g4_gwangju_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,광주광역시,합계,1014620,469316,18461,157756,288501,0,464718,4598,545304
1,광주광역시,동구,90767,43236,2201,16085,24444,0,42730,506,47531
2,광주광역시,서구,221034,102566,3911,34849,62942,0,101702,864,118468


In [63]:
g4_gwangju_22.to_csv("temp2_2_governor_gwangju_4.csv", index=False, encoding="utf-8-sig")

## Daejeon

In [64]:
g4_daejeon.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_염홍철,득표수_2_한나라당_박성효,득표수_3_민주당_최기복,득표수_4_민주노동당_박춘호,득표수_5_국민중심당_남충희,득표수_7_한국의미래를준비하는당_고낙정,득표수_계,무효투표수,기권수
0,대전광역시,합계,1077468,532568,217273,231489,6442,14899,55231,2707,528041,4527,544900
1,대전광역시,동구,183206,87967,36078,35745,1262,2401,10736,685,86907,1060,95239
2,대전광역시,중구,203076,100902,39961,44780,1202,2324,11290,456,100013,889,102174
3,대전광역시,서구,367265,182493,73194,83617,2040,4025,17622,759,181257,1236,184772
4,대전광역시,유성구,164867,84058,36081,36053,873,2979,7204,354,83544,514,80809


In [65]:
show_column_info(g4_daejeon)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_열린우리당_염홍철
5: 득표수_2_한나라당_박성효
6: 득표수_3_민주당_최기복
7: 득표수_4_민주노동당_박춘호
8: 득표수_5_국민중심당_남충희
9: 득표수_7_한국의미래를준비하는당_고낙정
10: 득표수_계
11: 무효투표수
12: 기권수

총 열 개수: 13
총 행 개수: 6


### v2.1

In [66]:
g4_daejeon_21 = rename_columns_by_index(g4_daejeon, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_염홍철' -> '진보정당'
인덱스 5: '득표수_2_한나라당_박성효' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [67]:
g4_daejeon_21 = add_sum_column(g4_daejeon_21, 6, 9, '그외정당')


=== 합산할 열들 (인덱스 6~9) ===
6: 득표수_3_민주당_최기복
7: 득표수_4_민주노동당_박춘호
8: 득표수_5_국민중심당_남충희
9: 득표수_7_한국의미래를준비하는당_고낙정

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 14


In [68]:
g4_daejeon_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_민주당_최기복,득표수_4_민주노동당_박춘호,득표수_5_국민중심당_남충희,득표수_7_한국의미래를준비하는당_고낙정,득표수_계,무효투표수,기권수,그외정당
0,대전광역시,합계,1077468,532568,217273,231489,6442,14899,55231,2707,528041,4527,544900,79279
1,대전광역시,동구,183206,87967,36078,35745,1262,2401,10736,685,86907,1060,95239,15084
2,대전광역시,중구,203076,100902,39961,44780,1202,2324,11290,456,100013,889,102174,15272


In [69]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g4_daejeon_21 = g4_daejeon_21[selected_columns].copy()

g4_daejeon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,대전광역시,합계,1077468,532568,231489,217273,79279,528041,4527,544900
1,대전광역시,동구,183206,87967,35745,36078,15084,86907,1060,95239
2,대전광역시,중구,203076,100902,44780,39961,15272,100013,889,102174


In [70]:
g4_daejeon_21.to_csv("temp2_1_governor_daejeon_4.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [71]:
g4_daejeon_22 = rename_columns_by_index(g4_daejeon, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_염홍철' -> '진보정당'
인덱스 5: '득표수_2_한나라당_박성효' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [72]:
g4_daejeon_22 = add_sum_column(g4_daejeon_22, 6, 9, '그외정당')
g4_daejeon_22 = add_zero_column(g4_daejeon_22, '무소속')


=== 합산할 열들 (인덱스 6~9) ===
6: 득표수_3_민주당_최기복
7: 득표수_4_민주노동당_박춘호
8: 득표수_5_국민중심당_남충희
9: 득표수_7_한국의미래를준비하는당_고낙정

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 14


In [73]:
g4_daejeon_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_민주당_최기복,득표수_4_민주노동당_박춘호,득표수_5_국민중심당_남충희,득표수_7_한국의미래를준비하는당_고낙정,득표수_계,무효투표수,기권수,그외정당,무소속
0,대전광역시,합계,1077468,532568,217273,231489,6442,14899,55231,2707,528041,4527,544900,79279,0
1,대전광역시,동구,183206,87967,36078,35745,1262,2401,10736,685,86907,1060,95239,15084,0
2,대전광역시,중구,203076,100902,39961,44780,1202,2324,11290,456,100013,889,102174,15272,0


In [74]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g4_daejeon_22 = g4_daejeon_22[selected_columns].copy()

g4_daejeon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,대전광역시,합계,1077468,532568,231489,217273,79279,0,528041,4527,544900
1,대전광역시,동구,183206,87967,35745,36078,15084,0,86907,1060,95239
2,대전광역시,중구,203076,100902,44780,39961,15272,0,100013,889,102174


In [75]:
g4_daejeon_22.to_csv("temp2_2_governor_daejeon_4.csv", index=False, encoding="utf-8-sig")

## Ulsan

In [76]:
g4_ulsan.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_심규명,득표수_2_한나라당_박맹우,득표수_4_민주노동당_노옥희,득표수_계,무효투표수,기권수
0,울산광역시,합계,790289,417200,47579,261361,104384,413324,3876,373089
1,울산광역시,중구,175337,88549,9023,59499,19390,87912,637,86788
2,울산광역시,남구,250864,123837,14533,85037,23446,123016,821,127027
3,울산광역시,동구,136957,75733,8675,40077,26290,75042,691,61224
4,울산광역시,북구,99489,55385,5176,28677,21047,54900,485,44104


In [77]:
show_column_info(g4_ulsan)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_열린우리당_심규명
5: 득표수_2_한나라당_박맹우
6: 득표수_4_민주노동당_노옥희
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 6


### v2.1

In [78]:
g4_ulsan_21 = rename_columns_by_index(g4_ulsan, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_심규명' -> '진보정당'
인덱스 5: '득표수_2_한나라당_박맹우' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [79]:
g4_ulsan_21 = add_sum_column(g4_ulsan_21, 6, 6, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_4_민주노동당_노옥희

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [80]:
g4_ulsan_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_4_민주노동당_노옥희,득표수_계,무효투표수,기권수,그외정당
0,울산광역시,합계,790289,417200,47579,261361,104384,413324,3876,373089,104384
1,울산광역시,중구,175337,88549,9023,59499,19390,87912,637,86788,19390
2,울산광역시,남구,250864,123837,14533,85037,23446,123016,821,127027,23446


In [81]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g4_ulsan_21 = g4_ulsan_21[selected_columns].copy()

g4_ulsan_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,울산광역시,합계,790289,417200,261361,47579,104384,413324,3876,373089
1,울산광역시,중구,175337,88549,59499,9023,19390,87912,637,86788
2,울산광역시,남구,250864,123837,85037,14533,23446,123016,821,127027


In [82]:
g4_ulsan_21.to_csv("temp2_1_governor_ulsan_4.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [83]:
g4_ulsan_22 = rename_columns_by_index(g4_ulsan, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_심규명' -> '진보정당'
인덱스 5: '득표수_2_한나라당_박맹우' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [84]:
g4_ulsan_22 = add_sum_column(g4_ulsan_22, 6, 6, '그외정당')
g4_ulsan_22 = add_zero_column(g4_ulsan_22, '무소속')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_4_민주노동당_노옥희

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [85]:
g4_ulsan_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_4_민주노동당_노옥희,득표수_계,무효투표수,기권수,그외정당,무소속
0,울산광역시,합계,790289,417200,47579,261361,104384,413324,3876,373089,104384,0
1,울산광역시,중구,175337,88549,9023,59499,19390,87912,637,86788,19390,0
2,울산광역시,남구,250864,123837,14533,85037,23446,123016,821,127027,23446,0


In [86]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g4_ulsan_22 = g4_ulsan_22[selected_columns].copy()

g4_ulsan_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,울산광역시,합계,790289,417200,261361,47579,104384,0,413324,3876,373089
1,울산광역시,중구,175337,88549,59499,9023,19390,0,87912,637,86788
2,울산광역시,남구,250864,123837,85037,14533,23446,0,123016,821,127027


In [87]:
g4_ulsan_22.to_csv("temp2_2_governor_ulsan_4.csv", index=False, encoding="utf-8-sig")

## Gyeonggi

In [88]:
g4_gyeonggi.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_진대제,득표수_2_한나라당_김문수,득표수_3_민주당_최기복,득표수_4_민주노동당_김용한,득표수_계,무효투표수,기권수
0,경기도,합계,7918828,3695552,1124317,2181677,148409,201106,3655509,40043,4223276
1,경기도,수원시장안구,206015,98094,31861,56361,3366,5795,97383,711,107921
2,경기도,수원시권선구,217455,97485,31707,56267,2822,5838,96634,851,119970
3,경기도,수원시팔달구,166662,72030,22480,42829,2501,3520,71330,700,94632
4,경기도,수원시영통구,173454,82006,31247,45246,1653,3442,81588,418,91448


In [89]:
show_column_info(g4_gyeonggi)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_열린우리당_진대제
5: 득표수_2_한나라당_김문수
6: 득표수_3_민주당_최기복
7: 득표수_4_민주노동당_김용한
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 45


### v2.1

In [90]:
g4_gyeonggi_21 = rename_columns_by_index(g4_gyeonggi, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_진대제' -> '진보정당'
인덱스 5: '득표수_2_한나라당_김문수' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [91]:
g4_gyeonggi_21 = add_sum_column(g4_gyeonggi_21, 6, 7, '그외정당')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_민주당_최기복
7: 득표수_4_민주노동당_김용한

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [92]:
g4_gyeonggi_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_민주당_최기복,득표수_4_민주노동당_김용한,득표수_계,무효투표수,기권수,그외정당
0,경기도,합계,7918828,3695552,1124317,2181677,148409,201106,3655509,40043,4223276,349515
1,경기도,수원시장안구,206015,98094,31861,56361,3366,5795,97383,711,107921,9161
2,경기도,수원시권선구,217455,97485,31707,56267,2822,5838,96634,851,119970,8660


In [93]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g4_gyeonggi_21 = g4_gyeonggi_21[selected_columns].copy()

g4_gyeonggi_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,경기도,합계,7918828,3695552,2181677,1124317,349515,3655509,40043,4223276
1,경기도,수원시장안구,206015,98094,56361,31861,9161,97383,711,107921
2,경기도,수원시권선구,217455,97485,56267,31707,8660,96634,851,119970


In [94]:
g4_gyeonggi_21.to_csv("temp2_1_governor_gyeonggi_4.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [95]:
g4_gyeonggi_22 = rename_columns_by_index(g4_gyeonggi, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_진대제' -> '진보정당'
인덱스 5: '득표수_2_한나라당_김문수' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [96]:
g4_gyeonggi_22 = add_sum_column(g4_gyeonggi_22, 6, 7, '그외정당')
g4_gyeonggi_22 = add_zero_column(g4_gyeonggi_22, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_민주당_최기복
7: 득표수_4_민주노동당_김용한

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [97]:
g4_gyeonggi_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_민주당_최기복,득표수_4_민주노동당_김용한,득표수_계,무효투표수,기권수,그외정당,무소속
0,경기도,합계,7918828,3695552,1124317,2181677,148409,201106,3655509,40043,4223276,349515,0
1,경기도,수원시장안구,206015,98094,31861,56361,3366,5795,97383,711,107921,9161,0
2,경기도,수원시권선구,217455,97485,31707,56267,2822,5838,96634,851,119970,8660,0


In [98]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g4_gyeonggi_22 = g4_gyeonggi_22[selected_columns].copy()

g4_gyeonggi_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,경기도,합계,7918828,3695552,2181677,1124317,349515,0,3655509,40043,4223276
1,경기도,수원시장안구,206015,98094,56361,31861,9161,0,97383,711,107921
2,경기도,수원시권선구,217455,97485,56267,31707,8660,0,96634,851,119970


In [99]:
g4_gyeonggi_22.to_csv("temp2_2_governor_gyeonggi_4.csv", index=False, encoding="utf-8-sig")

## Gangwon

In [100]:
g4_gangwon.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_이창복,득표수_2_한나라당_김진선,득표수_3_민주당_유재규,득표수_5_국민중심당_유승규,득표수_계,무효투표수,기권수
0,강원도,합계,1160977,681633,148302,471613,29028,19383,668326,13307,479344
1,강원도,춘천시,192644,104177,36231,45125,11660,7673,100689,3488,88467
2,강원도,원주시,215770,106902,31345,70598,2397,1321,105661,1241,108868
3,강원도,강릉시,171138,92512,12467,74805,2064,1709,91045,1467,78626
4,강원도,동해시,74884,43269,4903,36395,803,586,42687,582,31615


In [101]:
show_column_info(g4_gangwon)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_열린우리당_이창복
5: 득표수_2_한나라당_김진선
6: 득표수_3_민주당_유재규
7: 득표수_5_국민중심당_유승규
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 19


### v2.1

In [102]:
g4_gangwon_21 = rename_columns_by_index(g4_gangwon, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_이창복' -> '진보정당'
인덱스 5: '득표수_2_한나라당_김진선' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [103]:
g4_gangwon_21 = add_sum_column(g4_gangwon_21, 6, 7, '그외정당')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_민주당_유재규
7: 득표수_5_국민중심당_유승규

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [104]:
g4_gangwon_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_민주당_유재규,득표수_5_국민중심당_유승규,득표수_계,무효투표수,기권수,그외정당
0,강원도,합계,1160977,681633,148302,471613,29028,19383,668326,13307,479344,48411
1,강원도,춘천시,192644,104177,36231,45125,11660,7673,100689,3488,88467,19333
2,강원도,원주시,215770,106902,31345,70598,2397,1321,105661,1241,108868,3718


In [105]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g4_gangwon_21 = g4_gangwon_21[selected_columns].copy()

g4_gangwon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,강원도,합계,1160977,681633,471613,148302,48411,668326,13307,479344
1,강원도,춘천시,192644,104177,45125,36231,19333,100689,3488,88467
2,강원도,원주시,215770,106902,70598,31345,3718,105661,1241,108868


In [106]:
g4_gangwon_21.to_csv("temp2_1_governor_gangwon_4.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [107]:
g4_gangwon_22 = rename_columns_by_index(g4_gangwon, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_이창복' -> '진보정당'
인덱스 5: '득표수_2_한나라당_김진선' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [108]:
g4_gangwon_22 = add_sum_column(g4_gangwon_22, 6, 7, '그외정당')
g4_gangwon_22 = add_zero_column(g4_gangwon_22, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_민주당_유재규
7: 득표수_5_국민중심당_유승규

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [109]:
g4_gangwon_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_민주당_유재규,득표수_5_국민중심당_유승규,득표수_계,무효투표수,기권수,그외정당,무소속
0,강원도,합계,1160977,681633,148302,471613,29028,19383,668326,13307,479344,48411,0
1,강원도,춘천시,192644,104177,36231,45125,11660,7673,100689,3488,88467,19333,0
2,강원도,원주시,215770,106902,31345,70598,2397,1321,105661,1241,108868,3718,0


In [110]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g4_gangwon_22 = g4_gangwon_22[selected_columns].copy()

g4_gangwon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,강원도,합계,1160977,681633,471613,148302,48411,0,668326,13307,479344
1,강원도,춘천시,192644,104177,45125,36231,19333,0,100689,3488,88467
2,강원도,원주시,215770,106902,70598,31345,3718,0,105661,1241,108868


In [111]:
g4_gangwon_22.to_csv("temp2_2_governor_gangwon_4.csv", index=False, encoding="utf-8-sig")

## Chungbuk

In [112]:
g4_chungbuk.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_한범덕,득표수_2_한나라당_정우택,득표수_4_민주노동당_배창호,득표수_5_국민중심당_조병세,득표수_계,무효투표수,기권수
0,충청북도,합계,1126282,616053,185426,361157,39095,19646,605324,10729,510229
1,충청북도,청주시상당구,188493,89032,28260,52776,5015,2114,88165,867,99461
2,충청북도,청주시흥덕구,269249,121820,38846,70472,8623,2740,120681,1139,147429
3,충청북도,충주시,155769,83466,24287,50932,4746,2095,82060,1406,72303
4,충청북도,제천시,105920,60200,16169,37174,3981,1845,59169,1031,45720


In [113]:
show_column_info(g4_chungbuk)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_열린우리당_한범덕
5: 득표수_2_한나라당_정우택
6: 득표수_4_민주노동당_배창호
7: 득표수_5_국민중심당_조병세
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 14


### v2.1

In [114]:
g4_chungbuk_21 = rename_columns_by_index(g4_chungbuk, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_한범덕' -> '진보정당'
인덱스 5: '득표수_2_한나라당_정우택' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [115]:
g4_chungbuk_21 = add_sum_column(g4_chungbuk_21, 6, 7, '그외정당')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_4_민주노동당_배창호
7: 득표수_5_국민중심당_조병세

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [116]:
g4_chungbuk_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_4_민주노동당_배창호,득표수_5_국민중심당_조병세,득표수_계,무효투표수,기권수,그외정당
0,충청북도,합계,1126282,616053,185426,361157,39095,19646,605324,10729,510229,58741
1,충청북도,청주시상당구,188493,89032,28260,52776,5015,2114,88165,867,99461,7129
2,충청북도,청주시흥덕구,269249,121820,38846,70472,8623,2740,120681,1139,147429,11363


In [117]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g4_chungbuk_21 = g4_chungbuk_21[selected_columns].copy()

g4_chungbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,충청북도,합계,1126282,616053,361157,185426,58741,605324,10729,510229
1,충청북도,청주시상당구,188493,89032,52776,28260,7129,88165,867,99461
2,충청북도,청주시흥덕구,269249,121820,70472,38846,11363,120681,1139,147429


In [118]:
g4_chungbuk_21.to_csv("temp2_1_governor_chungbuk_4.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [119]:
g4_chungbuk_22 = rename_columns_by_index(g4_chungbuk, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_한범덕' -> '진보정당'
인덱스 5: '득표수_2_한나라당_정우택' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [120]:
g4_chungbuk_22 = add_sum_column(g4_chungbuk_22, 6, 7, '그외정당')
g4_chungbuk_22 = add_zero_column(g4_chungbuk_22, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_4_민주노동당_배창호
7: 득표수_5_국민중심당_조병세

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [121]:
g4_chungbuk_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_4_민주노동당_배창호,득표수_5_국민중심당_조병세,득표수_계,무효투표수,기권수,그외정당,무소속
0,충청북도,합계,1126282,616053,185426,361157,39095,19646,605324,10729,510229,58741,0
1,충청북도,청주시상당구,188493,89032,28260,52776,5015,2114,88165,867,99461,7129,0
2,충청북도,청주시흥덕구,269249,121820,38846,70472,8623,2740,120681,1139,147429,11363,0


In [122]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g4_chungbuk_22 = g4_chungbuk_22[selected_columns].copy()

g4_chungbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,충청북도,합계,1126282,616053,361157,185426,58741,0,605324,10729,510229
1,충청북도,청주시상당구,188493,89032,52776,28260,7129,0,88165,867,99461
2,충청북도,청주시흥덕구,269249,121820,70472,38846,11363,0,120681,1139,147429


In [123]:
g4_chungbuk_22.to_csv("temp2_2_governor_chungbuk_4.csv", index=False, encoding="utf-8-sig")

## Chungnam

In [124]:
g4_chungnam.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_오영교,득표수_2_한나라당_이완구,득표수_4_민주노동당_이용길,득표수_5_국민중심당_이명수,득표수_계,무효투표수,기권수
0,충청남도,합계,1503240,838462,178169,379420,52417,209254,819260,19202,664778
1,충청남도,천안시,372195,162726,36426,84436,15778,24136,160776,1950,209469
2,충청남도,공주시,100960,61082,13868,21948,3101,20682,59599,1483,39878
3,충청남도,보령시,84236,52539,16826,20818,2392,11282,51318,1221,31697
4,충청남도,아산시,156817,74771,10839,28561,6590,27467,73457,1314,82046


In [125]:
show_column_info(g4_chungnam)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_열린우리당_오영교
5: 득표수_2_한나라당_이완구
6: 득표수_4_민주노동당_이용길
7: 득표수_5_국민중심당_이명수
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 17


### v2.1

In [126]:
g4_chungnam_21 = rename_columns_by_index(g4_chungnam, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_오영교' -> '진보정당'
인덱스 5: '득표수_2_한나라당_이완구' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [127]:
g4_chungnam_21 = add_sum_column(g4_chungnam_21, 6, 7, '그외정당')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_4_민주노동당_이용길
7: 득표수_5_국민중심당_이명수

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [128]:
g4_chungnam_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_4_민주노동당_이용길,득표수_5_국민중심당_이명수,득표수_계,무효투표수,기권수,그외정당
0,충청남도,합계,1503240,838462,178169,379420,52417,209254,819260,19202,664778,261671
1,충청남도,천안시,372195,162726,36426,84436,15778,24136,160776,1950,209469,39914
2,충청남도,공주시,100960,61082,13868,21948,3101,20682,59599,1483,39878,23783


In [129]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g4_chungnam_21 = g4_chungnam_21[selected_columns].copy()

g4_chungnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,충청남도,합계,1503240,838462,379420,178169,261671,819260,19202,664778
1,충청남도,천안시,372195,162726,84436,36426,39914,160776,1950,209469
2,충청남도,공주시,100960,61082,21948,13868,23783,59599,1483,39878


In [130]:
g4_chungnam_21.to_csv("temp2_1_governor_chungnam_4.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [131]:
g4_chungnam_22 = rename_columns_by_index(g4_chungnam, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_오영교' -> '진보정당'
인덱스 5: '득표수_2_한나라당_이완구' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [132]:
g4_chungnam_22 = add_sum_column(g4_chungnam_22, 6, 7, '그외정당')
g4_chungnam_22 = add_zero_column(g4_chungnam_22, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_4_민주노동당_이용길
7: 득표수_5_국민중심당_이명수

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [133]:
g4_chungnam_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_4_민주노동당_이용길,득표수_5_국민중심당_이명수,득표수_계,무효투표수,기권수,그외정당,무소속
0,충청남도,합계,1503240,838462,178169,379420,52417,209254,819260,19202,664778,261671,0
1,충청남도,천안시,372195,162726,36426,84436,15778,24136,160776,1950,209469,39914,0
2,충청남도,공주시,100960,61082,13868,21948,3101,20682,59599,1483,39878,23783,0


In [134]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g4_chungnam_22 = g4_chungnam_22[selected_columns].copy()

g4_chungnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,충청남도,합계,1503240,838462,379420,178169,261671,0,819260,19202,664778
1,충청남도,천안시,372195,162726,84436,36426,39914,0,160776,1950,209469
2,충청남도,공주시,100960,61082,21948,13868,23783,0,59599,1483,39878


In [135]:
g4_chungnam_22.to_csv("temp2_2_governor_chungnam_4.csv", index=False, encoding="utf-8-sig")

## Jeonbuk

In [136]:
g4_jeonbuk.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_김완주,득표수_2_한나라당_문용주,득표수_3_민주당_정균환,득표수_4_민주노동당_염경석,득표수_계,무효투표수,기권수
0,전라북도,합계,1429632,827387,389436,62922,295891,61672,809921,17466,602245
1,전라북도,전주시완산구,252082,121291,59401,6322,45973,8406,120102,1189,130791
2,전라북도,전주시덕진구,198612,94137,45562,5075,34130,8560,93327,810,104475
3,전라북도,군산시,198467,111760,44385,19593,36122,9086,109186,2574,86707
4,전라북도,익산시,235847,123052,55008,8189,48019,9442,120658,2394,112795


In [137]:
show_column_info(g4_jeonbuk)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_열린우리당_김완주
5: 득표수_2_한나라당_문용주
6: 득표수_3_민주당_정균환
7: 득표수_4_민주노동당_염경석
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 16


### v2.1

In [138]:
g4_jeonbuk_21 = rename_columns_by_index(g4_jeonbuk, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_김완주' -> '진보정당'
인덱스 5: '득표수_2_한나라당_문용주' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [139]:
g4_jeonbuk_21 = add_sum_column(g4_jeonbuk_21, 6, 7, '그외정당')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_민주당_정균환
7: 득표수_4_민주노동당_염경석

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [140]:
g4_jeonbuk_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_민주당_정균환,득표수_4_민주노동당_염경석,득표수_계,무효투표수,기권수,그외정당
0,전라북도,합계,1429632,827387,389436,62922,295891,61672,809921,17466,602245,357563
1,전라북도,전주시완산구,252082,121291,59401,6322,45973,8406,120102,1189,130791,54379
2,전라북도,전주시덕진구,198612,94137,45562,5075,34130,8560,93327,810,104475,42690


In [141]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g4_jeonbuk_21 = g4_jeonbuk_21[selected_columns].copy()

g4_jeonbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,전라북도,합계,1429632,827387,62922,389436,357563,809921,17466,602245
1,전라북도,전주시완산구,252082,121291,6322,59401,54379,120102,1189,130791
2,전라북도,전주시덕진구,198612,94137,5075,45562,42690,93327,810,104475


In [142]:
g4_jeonbuk_21.to_csv("temp2_1_governor_jeonbuk_4.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [143]:
g4_jeonbuk_22 = rename_columns_by_index(g4_jeonbuk, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_김완주' -> '진보정당'
인덱스 5: '득표수_2_한나라당_문용주' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [144]:
g4_jeonbuk_22 = add_sum_column(g4_jeonbuk_22, 6, 7, '그외정당')
g4_jeonbuk_22 = add_zero_column(g4_jeonbuk_22, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_민주당_정균환
7: 득표수_4_민주노동당_염경석

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [145]:
g4_jeonbuk_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_민주당_정균환,득표수_4_민주노동당_염경석,득표수_계,무효투표수,기권수,그외정당,무소속
0,전라북도,합계,1429632,827387,389436,62922,295891,61672,809921,17466,602245,357563,0
1,전라북도,전주시완산구,252082,121291,59401,6322,45973,8406,120102,1189,130791,54379,0
2,전라북도,전주시덕진구,198612,94137,45562,5075,34130,8560,93327,810,104475,42690,0


In [146]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g4_jeonbuk_22 = g4_jeonbuk_22[selected_columns].copy()

g4_jeonbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,전라북도,합계,1429632,827387,62922,389436,357563,0,809921,17466,602245
1,전라북도,전주시완산구,252082,121291,6322,59401,54379,0,120102,1189,130791
2,전라북도,전주시덕진구,198612,94137,5075,45562,42690,0,93327,810,104475


In [147]:
g4_jeonbuk_22.to_csv("temp2_2_governor_jeonbuk_4.csv", index=False, encoding="utf-8-sig")

## Jeonnam

In [148]:
g4_jeonnam.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_서범석,득표수_2_한나라당_박재순,득표수_3_민주당_박준영,득표수_4_민주노동당_박웅두,득표수_계,무효투표수,기권수
0,전라남도,합계,1513912,973820,181756,55444,640894,68702,946796,27024,540092
1,전라남도,목포시,176932,88358,12717,2667,66179,5513,87076,1282,88574
2,전라남도,여수시,224001,122812,32665,4973,76454,6702,120794,2018,101189
3,전라남도,순천시,194227,105389,27461,4311,63894,7802,103468,1921,88838
4,전라남도,나주시,79340,55195,5955,2484,40865,4087,53391,1804,24145


In [149]:
show_column_info(g4_jeonnam)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_열린우리당_서범석
5: 득표수_2_한나라당_박재순
6: 득표수_3_민주당_박준영
7: 득표수_4_민주노동당_박웅두
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 23


### v2.1

In [150]:
g4_jeonnam_21 = rename_columns_by_index(g4_jeonnam, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_서범석' -> '진보정당'
인덱스 5: '득표수_2_한나라당_박재순' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [151]:
g4_jeonnam_21 = add_sum_column(g4_jeonnam_21, 6, 7, '그외정당')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_민주당_박준영
7: 득표수_4_민주노동당_박웅두

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [152]:
g4_jeonnam_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_민주당_박준영,득표수_4_민주노동당_박웅두,득표수_계,무효투표수,기권수,그외정당
0,전라남도,합계,1513912,973820,181756,55444,640894,68702,946796,27024,540092,709596
1,전라남도,목포시,176932,88358,12717,2667,66179,5513,87076,1282,88574,71692
2,전라남도,여수시,224001,122812,32665,4973,76454,6702,120794,2018,101189,83156


In [153]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g4_jeonnam_21 = g4_jeonnam_21[selected_columns].copy()

g4_jeonnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,전라남도,합계,1513912,973820,55444,181756,709596,946796,27024,540092
1,전라남도,목포시,176932,88358,2667,12717,71692,87076,1282,88574
2,전라남도,여수시,224001,122812,4973,32665,83156,120794,2018,101189


In [154]:
g4_jeonnam_21.to_csv("temp2_1_governor_jeonnam_4.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [155]:
g4_jeonnam_22 = rename_columns_by_index(g4_jeonnam, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_서범석' -> '진보정당'
인덱스 5: '득표수_2_한나라당_박재순' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [156]:
g4_jeonnam_22 = add_sum_column(g4_jeonnam_22, 6, 7, '그외정당')
g4_jeonnam_22 = add_zero_column(g4_jeonnam_22, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_민주당_박준영
7: 득표수_4_민주노동당_박웅두

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [157]:
g4_jeonnam_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_3_민주당_박준영,득표수_4_민주노동당_박웅두,득표수_계,무효투표수,기권수,그외정당,무소속
0,전라남도,합계,1513912,973820,181756,55444,640894,68702,946796,27024,540092,709596,0
1,전라남도,목포시,176932,88358,12717,2667,66179,5513,87076,1282,88574,71692,0
2,전라남도,여수시,224001,122812,32665,4973,76454,6702,120794,2018,101189,83156,0


In [158]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g4_jeonnam_22 = g4_jeonnam_22[selected_columns].copy()

g4_jeonnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,전라남도,합계,1513912,973820,55444,181756,709596,0,946796,27024,540092
1,전라남도,목포시,176932,88358,2667,12717,71692,0,87076,1282,88574
2,전라남도,여수시,224001,122812,4973,32665,83156,0,120794,2018,101189


In [159]:
g4_jeonnam_22.to_csv("temp2_2_governor_jeonnam_4.csv", index=False, encoding="utf-8-sig")

## Gyeongbuk

In [160]:
g4_gyeongbuk.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_박명재,득표수_2_한나라당_김관용,득표수_계,무효투표수,기권수
0,경상북도,합계,2087709,1284342,290358,961363,1251721,32621,803367
1,경상북도,포항시북구,190449,105458,25052,78490,103542,1916,84991
2,경상북도,포항시남구,191369,106552,28285,76190,104475,2077,84817
3,경상북도,울릉군,8429,7197,1978,5034,7012,185,1232
4,경상북도,경주시,210980,126368,24020,99188,123208,3160,84612


In [161]:
show_column_info(g4_gyeongbuk)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_열린우리당_박명재
5: 득표수_2_한나라당_김관용
6: 득표수_계
7: 무효투표수
8: 기권수

총 열 개수: 9
총 행 개수: 25


### v2.1 (XX)

In [162]:
g4_gyeongbuk_21 = rename_columns_by_index(g4_gyeongbuk, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_박명재' -> '진보정당'
인덱스 5: '득표수_2_한나라당_김관용' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [163]:
g4_gyeongbuk_21 = add_zero_column(g4_gyeongbuk_21, '그외정당')

In [164]:
g4_gyeongbuk_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_계,무효투표수,기권수,그외정당
0,경상북도,합계,2087709,1284342,290358,961363,1251721,32621,803367,0
1,경상북도,포항시북구,190449,105458,25052,78490,103542,1916,84991,0
2,경상북도,포항시남구,191369,106552,28285,76190,104475,2077,84817,0


In [165]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g4_gyeongbuk_21 = g4_gyeongbuk_21[selected_columns].copy()

g4_gyeongbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,경상북도,합계,2087709,1284342,961363,290358,0,1251721,32621,803367
1,경상북도,포항시북구,190449,105458,78490,25052,0,103542,1916,84991
2,경상북도,포항시남구,191369,106552,76190,28285,0,104475,2077,84817


In [166]:
g4_gyeongbuk_21.to_csv("temp2_1_governor_gyeongbuk_4.csv", index=False, encoding="utf-8-sig")

### v2.2 (XX,X)

In [167]:
g4_gyeongbuk_22 = rename_columns_by_index(g4_gyeongbuk, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_박명재' -> '진보정당'
인덱스 5: '득표수_2_한나라당_김관용' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [168]:
g4_gyeongbuk_22 = add_zero_column(g4_gyeongbuk_22, '그외정당')
g4_gyeongbuk_22 = add_zero_column(g4_gyeongbuk_22, '무소속')

In [169]:
g4_gyeongbuk_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_계,무효투표수,기권수,그외정당,무소속
0,경상북도,합계,2087709,1284342,290358,961363,1251721,32621,803367,0,0
1,경상북도,포항시북구,190449,105458,25052,78490,103542,1916,84991,0,0
2,경상북도,포항시남구,191369,106552,28285,76190,104475,2077,84817,0,0


In [170]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g4_gyeongbuk_22 = g4_gyeongbuk_22[selected_columns].copy()

g4_gyeongbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,경상북도,합계,2087709,1284342,961363,290358,0,0,1251721,32621,803367
1,경상북도,포항시북구,190449,105458,78490,25052,0,0,103542,1916,84991
2,경상북도,포항시남구,191369,106552,76190,28285,0,0,104475,2077,84817


In [171]:
g4_gyeongbuk_22.to_csv("temp2_2_governor_gyeongbuk_4.csv", index=False, encoding="utf-8-sig")

## Gyeongnam

In [172]:
g4_gyeongnam.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_김두관,득표수_2_한나라당_김태호,득표수_4_민주노동당_문성현,득표수_5_국민중심당_김재주,득표수_계,무효투표수,기권수
0,경상남도,합계,2375265,1373815,343137,852377,135823,18906,1350243,23572,1001450
1,경상남도,창원시,356493,184115,38627,112369,30193,1431,182620,1495,172378
2,경상남도,마산시,326663,171791,31539,122814,14066,1476,169895,1896,154872
3,경상남도,진주시,252537,145232,50737,77772,12912,1691,143112,2120,107305
4,경상남도,진해시,118111,70223,15379,46643,6344,751,69117,1106,47888


In [173]:
show_column_info(g4_gyeongnam)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_열린우리당_김두관
5: 득표수_2_한나라당_김태호
6: 득표수_4_민주노동당_문성현
7: 득표수_5_국민중심당_김재주
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 21


### v2.1

In [174]:
g4_gyeongnam_21 = rename_columns_by_index(g4_gyeongnam, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_김두관' -> '진보정당'
인덱스 5: '득표수_2_한나라당_김태호' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [175]:
g4_gyeongnam_21 = add_sum_column(g4_gyeongnam_21, 6, 7, '그외정당')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_4_민주노동당_문성현
7: 득표수_5_국민중심당_김재주

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [176]:
g4_gyeongnam_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_4_민주노동당_문성현,득표수_5_국민중심당_김재주,득표수_계,무효투표수,기권수,그외정당
0,경상남도,합계,2375265,1373815,343137,852377,135823,18906,1350243,23572,1001450,154729
1,경상남도,창원시,356493,184115,38627,112369,30193,1431,182620,1495,172378,31624
2,경상남도,마산시,326663,171791,31539,122814,14066,1476,169895,1896,154872,15542


In [177]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g4_gyeongnam_21 = g4_gyeongnam_21[selected_columns].copy()

g4_gyeongnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,경상남도,합계,2375265,1373815,852377,343137,154729,1350243,23572,1001450
1,경상남도,창원시,356493,184115,112369,38627,31624,182620,1495,172378
2,경상남도,마산시,326663,171791,122814,31539,15542,169895,1896,154872


In [178]:
g4_gyeongnam_21.to_csv("temp2_1_governor_gyeongnam_4.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [179]:
g4_gyeongnam_22 = rename_columns_by_index(g4_gyeongnam, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_김두관' -> '진보정당'
인덱스 5: '득표수_2_한나라당_김태호' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [180]:
g4_gyeongnam_22 = add_sum_column(g4_gyeongnam_22, 6, 7, '그외정당')
g4_gyeongnam_22 = add_zero_column(g4_gyeongnam_22, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_4_민주노동당_문성현
7: 득표수_5_국민중심당_김재주

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [181]:
g4_gyeongnam_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_4_민주노동당_문성현,득표수_5_국민중심당_김재주,득표수_계,무효투표수,기권수,그외정당,무소속
0,경상남도,합계,2375265,1373815,343137,852377,135823,18906,1350243,23572,1001450,154729,0
1,경상남도,창원시,356493,184115,38627,112369,30193,1431,182620,1495,172378,31624,0
2,경상남도,마산시,326663,171791,31539,122814,14066,1476,169895,1896,154872,15542,0


In [182]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g4_gyeongnam_22 = g4_gyeongnam_22[selected_columns].copy()

g4_gyeongnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,경상남도,합계,2375265,1373815,852377,343137,154729,0,1350243,23572,1001450
1,경상남도,창원시,356493,184115,112369,38627,31624,0,182620,1495,172378
2,경상남도,마산시,326663,171791,122814,31539,15542,0,169895,1896,154872


In [183]:
g4_gyeongnam_22.to_csv("temp2_2_governor_gyeongnam_4.csv", index=False, encoding="utf-8-sig")

## Jeju

In [184]:
g4_jeju.head()

,시도,구시군,선거인수,투표수,득표수_1_열린우리당_진철훈,득표수_2_한나라당_현명관,득표수_6_무소속_김태환,득표수_계,무효투표수,기권수
0,제주특별자치도,합계,411862,277003,44334,112774,117244,274352,2651,134859
1,제주특별자치도,제주시,218768,138717,22748,56423,58722,137893,824,80051
2,제주특별자치도,북제주군,74387,51650,10793,16607,23455,50855,795,22737
3,제주특별자치도,서귀포시,62209,43561,5087,19930,18098,43115,446,18648
4,제주특별자치도,남제주군,56498,43075,5706,19814,16969,42489,586,13423


In [185]:
show_column_info(g4_jeju)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_열린우리당_진철훈
5: 득표수_2_한나라당_현명관
6: 득표수_6_무소속_김태환
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 5


### v2.1

In [186]:
g4_jeju_21 = rename_columns_by_index(g4_jeju, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_진철훈' -> '진보정당'
인덱스 5: '득표수_2_한나라당_현명관' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [187]:
g4_jeju_21 = add_sum_column(g4_jeju_21, 6, 6, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_6_무소속_김태환

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [188]:
g4_jeju_21.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_6_무소속_김태환,득표수_계,무효투표수,기권수,그외정당
0,제주특별자치도,합계,411862,277003,44334,112774,117244,274352,2651,134859,117244
1,제주특별자치도,제주시,218768,138717,22748,56423,58722,137893,824,80051,58722
2,제주특별자치도,북제주군,74387,51650,10793,16607,23455,50855,795,22737,23455


In [189]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g4_jeju_21 = g4_jeju_21[selected_columns].copy()

g4_jeju_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,제주특별자치도,합계,411862,277003,112774,44334,117244,274352,2651,134859
1,제주특별자치도,제주시,218768,138717,56423,22748,58722,137893,824,80051
2,제주특별자치도,북제주군,74387,51650,16607,10793,23455,50855,795,22737


In [190]:
g4_jeju_21.to_csv("temp2_1_governor_jeju_4.csv", index=False, encoding="utf-8-sig")

### v2.2 (XX)

In [191]:
g4_jeju_22 = rename_columns_by_index(g4_jeju, [4, 5], ['진보정당', '보수정당'])

인덱스 4: '득표수_1_열린우리당_진철훈' -> '진보정당'
인덱스 5: '득표수_2_한나라당_현명관' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [192]:
g4_jeju_22 = add_sum_column(g4_jeju_22, 6, 6, '무소속')
g4_jeju_22 = add_zero_column(g4_jeju_22, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_6_무소속_김태환

=== 새로운 열 추가 완료 ===
새로운 열 이름: 무소속
총 열 개수: 11


In [193]:
g4_jeju_22.head(3)

,시도,구시군,선거인수,투표수,진보정당,보수정당,득표수_6_무소속_김태환,득표수_계,무효투표수,기권수,무소속,그외정당
0,제주특별자치도,합계,411862,277003,44334,112774,117244,274352,2651,134859,117244,0
1,제주특별자치도,제주시,218768,138717,22748,56423,58722,137893,824,80051,58722,0
2,제주특별자치도,북제주군,74387,51650,10793,16607,23455,50855,795,22737,23455,0


In [194]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g4_jeju_22 = g4_jeju_22[selected_columns].copy()

g4_jeju_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,제주특별자치도,합계,411862,277003,112774,44334,0,117244,274352,2651,134859
1,제주특별자치도,제주시,218768,138717,56423,22748,0,58722,137893,824,80051
2,제주특별자치도,북제주군,74387,51650,16607,10793,0,23455,50855,795,22737


In [195]:
g4_jeju_22.to_csv("temp2_2_governor_jeju_4.csv", index=False, encoding="utf-8-sig")

# Batch CSV Files to ZIP

In [196]:
import zipfile
import glob

# Find all CSV files in current directory
csv_files = glob.glob('*.csv')

# Create ZIP file
with zipfile.ZipFile('all_csv_files.zip', 'w') as zipf:
   for file in csv_files:
       zipf.write(file)
       print(f"Added: {file}")  # Show progress

print(f"Total {len(csv_files)} files compressed.")

Added: temp2_1_governor_incheon_4.csv
Added: temp2_2_governor_ulsan_4.csv
Added: temp2_1_governor_gyeongbuk_4.csv
Added: temp2_1_governor_gyeongnam_4.csv
Added: temp2_1_governor_chungnam_4.csv
Added: temp2_2_governor_gangwon_4.csv
Added: temp2_2_governor_seoul_4.csv
Added: temp2_1_governor_jeju_4.csv
Added: temp2_2_governor_gyeonggi_4.csv
Added: temp2_1_governor_chungbuk_4.csv
Added: temp2_1_governor_ulsan_4.csv
Added: temp2_1_governor_gyeonggi_4.csv
Added: temp2_2_governor_gyeongnam_4.csv
Added: temp2_1_governor_gangwon_4.csv
Added: temp2_1_governor_seoul_4.csv
Added: temp2_2_governor_jeonnam_4.csv
Added: temp2_2_governor_gyeongbuk_4.csv
Added: temp2_2_governor_busan_4.csv
Added: temp2_1_governor_jeonnam_4.csv
Added: temp2_2_governor_jeju_4.csv
Added: temp2_2_governor_incheon_4.csv
Added: temp2_1_governor_gwangju_4.csv
Added: temp2_1_governor_daegu_4.csv
Added: temp2_2_governor_jeonbuk_4.csv
Added: temp2_2_governor_daejeon_4.csv
Added: temp2_2_governor_chungnam_4.csv
Added: temp2_2_go